In [25]:
import pandas as pd
import numpy as np
from selenium import webdriver
import requests
import bs4
from bs4 import BeautifulSoup
import time

In [26]:
qualifying_results = pd.DataFrame()
for year in list(range(2014,2022)):
    url = 'https://www.formula1.com/en/results.html/{}/races.html'
    r = requests.get(url.format(year))
    soup = BeautifulSoup(r.text, 'html.parser')
    
    year_links = []
    for page in soup.find_all('a', attrs = {'class':"resultsarchive-filter-item-link FilterTrigger"}):
        link = page.get('href')
        if f'/en/results.html/{year}/races/' in link: 
            year_links.append(link)

    year_df = pd.DataFrame()
    new_url = 'https://www.formula1.com{}'
    for n, link in list(enumerate(year_links)):
        link = link.replace('race-result.html', 'starting-grid.html')
        df = pd.read_html(new_url.format(link))
        df = df[0]
        df['season'] = year
        df['round'] = n+1
        for col in df:
            if 'Unnamed' in col:
                df.drop(col, axis = 1, inplace = True)

        year_df = pd.concat([year_df, df])

    qualifying_results = pd.concat([qualifying_results, year_df])
    
print(qualifying_results.shape)

(3229, 7)


In [27]:
qualifying_results.head()

,Pos,No,Driver,Car,Time,season,round
0,1,44,Lewis Hamilton HAM,Mercedes,1:44.231,2014,1
1,2,3,Daniel Ricciardo RIC,Red Bull Racing Renault,1:44.548,2014,1
2,3,6,Nico Rosberg ROS,Mercedes,1:44.595,2014,1
3,4,20,Kevin Magnussen MAG,McLaren Mercedes,1:45.745,2014,1
4,5,14,Fernando Alonso ALO,Ferrari,1:45.819,2014,1


In [28]:
qualifying_results.tail()

,Pos,No,Driver,Car,Time,season,round
14,15,5,Sebastian Vettel VET,Aston Martin Mercedes,1:24.305,2021,22
15,16,6,Nicholas Latifi LAT,Williams Mercedes,1:24.338,2021,22
16,17,63,George Russell RUS,Williams Mercedes,1:24.423,2021,22
17,18,7,Kimi Räikkönen RAI,Alfa Romeo Racing Ferrari,1:24.779,2021,22
18,19,47,Mick Schumacher MSC,Haas Ferrari,1:24.906,2021,22


In [29]:
qualifying_results.rename(columns = {'Pos': 'grid_position', 'Driver': 'driver_name', 'Car': 'car',
                                     'Time': 'qualifying_time'}, inplace = True)

In [30]:
qualifying_results.drop('No',axis=1)

,grid_position,driver_name,car,qualifying_time,season,round
0,1,Lewis Hamilton HAM,Mercedes,1:44.231,2014,1
1,2,Daniel Ricciardo RIC,Red Bull Racing Renault,1:44.548,2014,1
2,3,Nico Rosberg ROS,Mercedes,1:44.595,2014,1
3,4,Kevin Magnussen MAG,McLaren Mercedes,1:45.745,2014,1
4,5,Fernando Alonso ALO,Ferrari,1:45.819,2014,1
...,...,...,...,...,...,...
14,15,Sebastian Vettel VET,Aston Martin Mercedes,1:24.305,2021,22
15,16,Nicholas Latifi LAT,Williams Mercedes,1:24.338,2021,22
16,17,George Russell RUS,Williams Mercedes,1:24.423,2021,22
17,18,Kimi Räikkönen RAI,Alfa Romeo Racing Ferrari,1:24.779,2021,22


In [31]:
qualifying_results

,grid_position,No,driver_name,car,qualifying_time,season,round
0,1,44,Lewis Hamilton HAM,Mercedes,1:44.231,2014,1
1,2,3,Daniel Ricciardo RIC,Red Bull Racing Renault,1:44.548,2014,1
2,3,6,Nico Rosberg ROS,Mercedes,1:44.595,2014,1
3,4,20,Kevin Magnussen MAG,McLaren Mercedes,1:45.745,2014,1
4,5,14,Fernando Alonso ALO,Ferrari,1:45.819,2014,1
...,...,...,...,...,...,...,...
14,15,5,Sebastian Vettel VET,Aston Martin Mercedes,1:24.305,2021,22
15,16,6,Nicholas Latifi LAT,Williams Mercedes,1:24.338,2021,22
16,17,63,George Russell RUS,Williams Mercedes,1:24.423,2021,22
17,18,7,Kimi Räikkönen RAI,Alfa Romeo Racing Ferrari,1:24.779,2021,22


In [32]:
qualifying_results.to_csv('qualifying.csv', index = False)